In [1]:
from arcgis.gis import GIS
gis=GIS("home")

In [14]:
#import necessary libraries
import io
import arcpy
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.widgets import MapView
from zipfile import ZipFile
import os
from osgeo import ogr
from arcgis.geometry import SpatialReference

In [3]:
#create a function to request and download data from Minnesota GeoCommons

#get the "mn county boundaries" package from MN Geocommons
mngc_res = requests.get("https://gisdata.mn.gov/api/3/action/package_show?id=d199886f-474f-4fe0-aba4-c7071ab92a35")  

#get the MN Geocommons response as a json
mngc_json_out = mngc_res.json()
print(mngc_json_out["result"]["resources"])
##get the actual zipfile url from the directory in mn geocommons- this is a line  within a list within a dictionary within a dictionary, show that url so it can be checked
url_mngc = mngc_json_out["result"]["resources"][2]["url"]
print(url_mngc)


[{'mimetype': None, 'cache_url': None, 'hash': '', 'description': '', 'gdrsResGuid': '{24e4ec6f-42fc-4195-9498-066f89f6c885}', 'cache_last_updated': None, 'url': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_counties/fgdb_bdry_counties.zip', 'name': 'ESRI File Geodatabase', 'format': 'fgdb', 'package_id': 'd199886f-474f-4fe0-aba4-c7071ab92a35', 'created': '2024-10-03T08:02:42.413613', 'state': 'active', 'mimetype_inner': None, 'last_modified': None, 'position': 0, 'revision_id': 'b2728ce6-c4c2-4b70-997b-a6adc38b2574', 'url_type': None, 'id': '9ee754fc-735e-402b-aff9-ebf09d67e45b', 'resource_type': 'fgdb', 'size': None}, {'mimetype': None, 'cache_url': None, 'hash': '', 'description': '', 'gdrsResGuid': '{2e998e84-1c25-4c41-ba66-0fe9e05ca8e2}', 'cache_last_updated': None, 'url': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_counties/gpkg_bdry_counties.zip', 'name': 'OGC GeoPackage', 'format': 'gpkg', 'package_id': 'd199886f-474f-4fe0

In [21]:
##get the files associated with the api request from the previous step as a zip file, show that you got them, save them in a folder
mngc_bdry_counties_zip = requests.get(url_mngc, allow_redirects=True)

open(r"C:\Users\rockx057\Downloads\mngc_bdry_counties.zip", 'wb').write(mngc_bdry_counties_zip.content)

with ZipFile(r"C:\Users\rockx057\Downloads\mngc_bdry_counties.zip", 'r') as mngc_bdry_counties:
    mngc_bdry_counties.extractall(
        path=r"C:\Users\rockx057\Downloads")


In [3]:
#read shapefile into spatially enabled dataframe- show the dataframe
df_mngc = pd.DataFrame.spatial.from_featureclass(r"C:\Users\rockx057\Downloads\County_Boundaries_in_Minnesota.shp")

print(df_mngc.head())
df_mngc

   FID  ...                                              SHAPE
0    0  ...  {"rings": [[[254267.48900000006, 5317428.819],...
1    1  ...  {"rings": [[[460790.98000000045, 5009684.501],...
2    2  ...  {"rings": [[[333638.16220000014, 5071100.27899...
3    3  ...  {"rings": [[[419160.2740000002, 5045580.789000...
4    4  ...  {"rings": [[[302069.1983000003, 4857932.176000...

[5 rows x 9 columns]


,FID,COUNTY_NAM,COUNTY_COD,COUNTY_FIP,COUNTY_GNI,ATP_CODE,SHAPE_Leng,SHAPE_Area,SHAPE
0,0,Red Lake,63,125,659508,2,160725.772889,1121198845.29,"{""rings"": [[[254267.48900000006, 5317428.819],..."
1,1,Hennepin,27,053,659472,M,189829.833881,1570490144.08,"{""rings"": [[[460790.98000000045, 5009684.501],..."
2,2,Stearns,73,145,659517,3,280898.930397,3598882111.63,"{""rings"": [[[333638.16220000014, 5071100.27899..."
3,3,Sherburne,71,141,659515,3,163652.509384,1166966708.25,"{""rings"": [[[419160.2740000002, 5045580.789000..."
4,4,Murray,51,101,659496,8,173803.04716,1864170293.52,"{""rings"": [[[302069.1983000003, 4857932.176000..."
...,...,...,...,...,...,...,...,...,...
82,82,Norman,54,107,659499,2,247097.098624,2272879542.21,"{""rings"": [[[223954.7450000001, 5267125.892000..."
83,83,Goodhue,25,049,659470,6,233063.11778,2020004256.93,"{""rings"": [[[530126.9800000004, 4941870.82], [..."
84,84,Dakota,19,037,659464,M,188056.213349,1517534633.81,"{""rings"": [[[492821.76999999955, 4974434.601],..."
85,85,Faribault,22,043,659467,7,173942.500141,1868579448.39,"{""rings"": [[[407752.21999999974, 4855630.841],..."


In [4]:
#reproject this into WGS 84
# make it a spatially enabled df
df_gecommons = pd.DataFrame.spatial.from_featureclass(r"C:\Users\rockx057\Downloads\County_Boundaries_in_Minnesota.shp")
df_gecommons.spatial.project(SpatialReference(4326))
df_gecommons

,FID,COUNTY_NAM,COUNTY_COD,COUNTY_FIP,COUNTY_GNI,ATP_CODE,SHAPE_Leng,SHAPE_Area,SHAPE
0,0,Red Lake,63,125,659508,2,160725.772889,1121198845.29,"{""rings"": [[[-96.2920295969504, 47.96295549646..."
1,1,Hennepin,27,053,659472,M,189829.833881,1570490144.08,"{""rings"": [[[-93.49956599190654, 45.2395607834..."
2,2,Stearns,73,145,659517,3,280898.930397,3598882111.63,"{""rings"": [[[-95.1397764088118, 45.77342266251..."
3,3,Sherburne,71,141,659515,3,163652.509384,1166966708.25,"{""rings"": [[[-94.03580931291634, 45.5590700155..."
4,4,Murray,51,101,659496,8,173803.04716,1864170293.52,"{""rings"": [[[-95.46245024888923, 43.8479460520..."
...,...,...,...,...,...,...,...,...,...
82,82,Norman,54,107,659499,2,247097.098624,2272879542.21,"{""rings"": [[[-96.66542793929916, 47.4989379190..."
83,83,Goodhue,25,049,659470,6,233063.11778,2020004256.93,"{""rings"": [[[-92.62019151129338, 44.6295610611..."
84,84,Dakota,19,037,659464,M,188056.213349,1517534633.81,"{""rings"": [[[-93.09095534911138, 44.9233046613..."
85,85,Faribault,22,043,659467,7,173942.500141,1868579448.39,"{""rings"": [[[-94.14766400467082, 43.8480001515..."


In [5]:
#create a function to request and download data from Esri
arc_url = ("https://webgis.dot.state.mn.us/65agsf1/rest/services/sdw_trans/ROUTES_TRUNK_HIGHWAY/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=4326&havingClause=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&returnTrueCurves=false&returnExceededLimitFeatures=false&quantizationParameters=&returnCentroid=false&timeReferenceUnknownClient=false&sqlFormat=none&resultType=&featureEncoding=esriDefault&datumTransformation=&f=geojson")
arc_response = requests.get(arc_url, allow_redirects = True)

##Convert response object into JSON
arc_gj = arc_response.json()

fs = FeatureSet.from_geojson(arc_gj)
sedf = fs.sdf

In [6]:
sedf

,ROUTE_NAME,OBJECTID,SHAPE
0,US 218-I,1,"{""paths"": [[[-92.94424965199994, 43.4997791680..."
1,MN 6-D,2,"{""paths"": [[[-93.86995927599997, 46.3448620900..."
2,MN 6-I,3,"{""paths"": [[[-93.86995927599997, 46.3448620900..."
3,MN 9-D,4,"{""paths"": [[[-94.93874822799995, 45.2917858190..."
4,MN 9-I,5,"{""paths"": [[[-94.93874822799995, 45.2917858190..."
...,...,...,...
380,MN 23-D,381,"{""paths"": [[[-96.43233509399994, 43.6080995770..."
381,UNM 67-I,382,"{""paths"": [[[-95.46050889199995, 44.7369864060..."
382,MN 7-I,383,"{""paths"": [[[-96.70003017999994, 45.5587314850..."
383,MN 23-I,384,"{""paths"": [[[-96.43233509399994, 43.6080995770..."


In [18]:
#create a function to request and download data from NDAWN


##request hourly data from 9/30 for Ada, MN (Station 78), 
ndawn_url = "https://ndawn.ndsu.nodak.edu/table.csv?ttype=hourly&station=78&begin_date=2024-09-30&end_date=2024-09-30"
ndawn_res = requests.get(ndawn_url, allow_redirects=True)

## turn CSV into dataframe, ignoring the text above the header row, and the units row
ndawn_csv_to_df = pd.read_csv(io.StringIO(ndawn_res.content.decode("utf-8")), header = 3, skiprows=[4])

ndawn_csv_to_df



,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Hour,Avg Air Temp,Avg Air Temp Flag,Avg Rel Hum,Avg Rel Hum Flag,Avg Bare Soil Temp,Avg Bare Soil Temp Flag,Avg Turf Soil Temp,Avg Turf Soil Temp Flag,Avg Wind Speed,Avg Wind Speed Flag,Max Wind Speed,Max Wind Speed Flag,Avg Wind Dir,Avg Wind Dir Flag,Avg Wind Dir SD,Avg Wind Dir SD Flag,Avg Sol Rad,Avg Sol Rad Flag,Total Rainfall,Total Rainfall Flag,Avg Baro Press,Avg Baro Press Flag,Avg Dew Point,Avg Dew Point Flag,Avg Wind Chill,Avg Wind Chill Flag,Avg Air Temp at 9 m,Avg Air Temp at 9 m Flag,Avg Rel Hum at 9 m,Avg Rel Hum at 9 m Flag,Avg Wind Speed at 10 m,Avg Wind Speed at 10 m Flag,Max Wind Speed at 10 m,Max Wind Speed at 10 m Flag,Avg Wind Dir at 10 m,Avg Wind Dir at 10 m Flag,Avg Wind Dir SD at 10 m,Avg Wind Dir SD at 10 m Flag
0,Ada,47.32119,-96.51406,910,2024,9,30,100,72.788,NaN,44.530,NaN,62.744,NaN,62.258,NaN,17.294,NaN,28.454,NaN,163.500,NaN,8.530,NaN,0.000,NaN,0.0,NaN,NaN,NaN,49.994,NaN,72.788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ada,47.32119,-96.51406,910,2024,9,30,200,72.554,NaN,42.320,NaN,62.492,NaN,62.132,NaN,17.918,NaN,32.682,NaN,165.500,NaN,9.300,NaN,0.000,NaN,0.0,NaN,NaN,NaN,48.412,NaN,72.554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ada,47.32119,-96.51406,910,2024,9,30,300,72.122,NaN,41.400,NaN,62.312,NaN,62.024,NaN,18.589,NaN,32.547,NaN,165.900,NaN,8.930,NaN,0.000,NaN,0.0,NaN,NaN,NaN,47.431,NaN,72.122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ada,47.32119,-96.51406,910,2024,9,30,400,72.968,NaN,43.020,NaN,62.114,NaN,61.898,NaN,21.497,NaN,36.775,NaN,168.200,NaN,9.280,NaN,0.000,NaN,0.0,NaN,NaN,NaN,49.230,NaN,72.968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ada,47.32119,-96.51406,910,2024,9,30,500,71.834,NaN,44.460,NaN,62.024,NaN,61.844,NaN,21.430,NaN,34.046,NaN,169.300,NaN,9.720,NaN,0.000,NaN,0.0,NaN,NaN,NaN,49.078,NaN,71.834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Ada,47.32119,-96.51406,910,2024,9,30,600,70.538,NaN,47.290,NaN,61.970,NaN,61.790,NaN,17.444,NaN,29.326,NaN,171.300,NaN,9.180,NaN,0.001,NaN,0.0,NaN,NaN,NaN,49.546,NaN,70.538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ada,47.32119,-96.51406,910,2024,9,30,700,72.428,NaN,46.170,NaN,61.862,NaN,61.736,NaN,19.573,NaN,31.049,NaN,179.200,NaN,9.150,NaN,1.299,NaN,0.0,NaN,NaN,NaN,50.640,NaN,72.428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Ada,47.32119,-96.51406,910,2024,9,30,800,70.772,NaN,50.200,NaN,61.934,NaN,61.736,NaN,20.311,NaN,36.149,NaN,186.300,NaN,9.840,NaN,10.464,NaN,0.0,NaN,NaN,NaN,51.376,NaN,70.772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Ada,47.32119,-96.51406,910,2024,9,30,900,72.140,NaN,49.440,NaN,62.258,NaN,61.772,NaN,17.406,NaN,29.438,NaN,208.800,NaN,13.530,NaN,24.566,NaN,0.0,NaN,NaN,NaN,52.231,NaN,72.140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Ada,47.32119,-96.51406,910,2024,9,30,1000,73.310,NaN,40.940,NaN,62.870,NaN,61.952,NaN,16.795,NaN,36.015,NaN,278.400,NaN,20.690,NaN,30.146,NaN,0.0,NaN,NaN,NaN,48.212,NaN,73.310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
##Norman county pull
norman_county = df_gecommons[df_gecommons["FID"] == 82]
norman_county

,FID,COUNTY_NAM,COUNTY_COD,COUNTY_FIP,COUNTY_GNI,ATP_CODE,SHAPE_Leng,SHAPE_Area,SHAPE
82,82,Norman,54,107,659499,2,247097.098624,2272879542.21,"{""rings"": [[[-96.66542793929916, 47.4989379190..."


#spatially join the data and print attribute table heading
# Create two SEDFs
NDAWN_sdf = GeoAccessor.from_xy(ndawn_csv_to_df, 'Longitude', 'Latitude', sr=4326)


# Perform the spatial join
joined_sdf = norman_county.spatial.join(NDAWN_sdf)

# Print the resulting SEDF
print(joined_sdf.head())

In [30]:
## export both dataframes to feature classes
NDAWN_sdf = pd.DataFrame.spatial.from_xy(ndawn_csv_to_df, 'Longitude', 'Latitude')
NDAWN_sdf[["SHAPE", "Station Name", "Avg Air Temp"]].spatial.to_featureclass(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 1 5571\Default.gdb\NDAWN_station")



'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 1 5571\\Default.gdb\\NDAWN_station'

In [26]:
norman_county.spatial.to_featureclass(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 1 5571\Default.gdb\Norman_County")

'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 1 5571\\Default.gdb\\Norman_County'

In [31]:
#spatially join the two feature classes, save to the geodatabase

arcpy.analysis.SpatialJoin("Norman_County", "NDAWN_station", r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 1 5571\Default.gdb\Joined_Ada_Norman",)

<Result 'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 1 5571\\Default.gdb\\Joined_Ada_Norman'>